In [1]:
#!pip install triton
import torch
import triton
import triton.language as tl
DEVICE = triton.runtime.driver.active.get_active_torch_device()
  



# GPU meaning: Many threads run together in lockstep groups on an SM.

@triton.jit
def add_kernel(x_ptr, n_elements, BLOCK: tl.constexpr):
    pid = tl.program_id(0)
    offsets = pid * BLOCK + tl.arange(0, BLOCK)
    mask = offsets < n_elements
    vals = tl.load(x_ptr + offsets, mask=mask, other=0.0)
    tl.device_print('pid=', pid, ' first_val=', vals[0])

size = 98432
x = torch.rand(size, device=DEVICE)
BLOCK = 128
grid = (triton.cdiv(size, BLOCK),)
add_kernel[grid](x, size, BLOCK=BLOCK)


In [7]:
# GPU meaning: Many threads run together in lockstep groups on an SM.
size = 98432
x_ptr = torch.rand(size, device=DEVICE)

@triton.jit
def add_kernel(x_ptr):

    pid = tl.program_id(0)
    offsets=pid * 128 + tl.arange(0, 128)
    tl.device_print('pid=', offsets)
    x=tl.load(x_ptr+offsets)
    y=x*2
                                 

add_kernel[(4,)](x_ptr)
